In [1]:
import mobster as mb


Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
data = mb.load_example_data("./")

In [3]:
fitted = mb.fit_mobster(data, 0,0, max_it = 500, lr = 0.01)

Running MOBSTER on 3 karyotypes with 0 subclones.
Fitting a model without tail
Computing cluster assignements.
Computing information criteria.
Done!


ELBO: inf  : 100%|██████████| 500/500 [00:11<00:00, 43.75it/s]         


In [4]:
mb.plot_results(data, fitted, output= "cane.png")